In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [2]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [3]:
from openai import OpenAI

client = OpenAI()

# 定义一个Prompt函数
def chat_completion(prompt, model="gpt-4", temperature=0):
    res = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    print(res.choices[0].message.content)

# Context, task and role

如果给大模型提供的Prompt包括Context, task and role，那么生成的效果往往会更好；当然如果任务特别复杂，也需要zero-shot，few-shot或者fine-tune才可以。

## Context

In [4]:
# 以下是较短的Prompt，提供较少的上下文信息，我们看一下生成的效果如何
chat_completion("Give me a suggestion for the main course for today's lunch.")

How about grilled chicken with roasted vegetables and quinoa? It's a healthy and delicious option.


In [5]:
# 在上述Prompt中提供更多的上下文信息，我们看一下生成的效果如何
prompt = """
Context: I do 2 hours of sport a day. I am vegetarian, and I don't like green
vegetables. I am conscientious about eating healthily.
Task: Give me a suggestion for a main course for today's lunch."""
chat_completion(prompt)

How about a hearty Chickpea Curry with brown rice? It's packed with protein and fiber from the chickpeas, and you can add any colorful vegetables you like such as bell peppers, tomatoes, or carrots. The curry spices will add a lot of flavor, and the brown rice will give you the energy you need for your sports activities.


很明显，提供丰富的上下文，模型会生成更多的内容

In [6]:
# 构建良好的Prompt是一个试错过程。在下面的示例中，我们不再要求模型回答问题，而是指示它以增强补全效果为目标来向我们提出上下文的问题。
prompt = """
Context: I do 2 hours of sport a day. I am vegetarian and I don't like green
vegetables. I am very careful to eat healthily.
Task: Give me a suggestion for a main course for today's lunch?
Do not perform the requested task! Instead, can you ask me questions about the
context so that when I answer, you can perform the requested task more
efficiently?
"""
chat_completion(prompt)

1. What types of sports do you engage in daily? This can help determine the amount of calories you need.
2. What are your favorite vegetarian foods?
3. Are there any specific dietary restrictions or allergies I should be aware of?
4. Do you prefer a hot or cold main course for lunch?
5. Are there any specific cuisines you enjoy?
6. Since you don't like green vegetables, are there other colors of vegetables you prefer?
7. Do you include dairy and eggs in your vegetarian diet?
8. How many meals do you usually have in a day? This can help determine the portion size for your lunch.


## Task

In [7]:
# 上面介绍了Prompt中Task中的基本应用，我们可以为该任务提供更多的细节来说明我们想要的内容，比如要求模型列出一张表格，列出每种食材所需的可数
prompt = """
Context: I do 2 hours of sport a day. I am vegetarian, and I don't like green
vegetables. I am conscientious about eating healthily.
Task: Give me a suggestion for a main course for today's lunch.
With this suggestion, I also want a table with two columns where each row
contains an ingredient from the main course.
The first column in the table is the name of the ingredient.
The second column of the table is the number of grams of that ingredient needed
for one person. Do not give the recipe for preparing the main course.
"""
chat_completion(prompt)

Main Course Suggestion: Vegetarian Quinoa Stir-Fry

| Ingredient Name | Quantity (in grams) |
|-----------------|---------------------|
| Quinoa          | 185                 |
| Olive Oil       | 15                  |
| Garlic          | 10                  |
| Onion           | 70                  |
| Bell Peppers    | 75                  |
| Carrots         | 60                  |
| Mushrooms       | 70                  |
| Tofu            | 150                 |
| Soy Sauce       | 30                  |
| Sesame Seeds    | 10                  |


## Role

In [8]:
# 添加角色可以更好的控制模型输出，比如我们赋予模型运动营养专家的角色，来看看生成的效果
prompt = """
Role: You are a nutritionist designing healthy diets for high-performance athletes. You take into account the nutrition needed for a good recovery.
Context: I do 2 hours of sport a day. I am vegetarian, and I don't like green vegetables. I am conscientious about eating healthily.
Task: Based on your expertise defined in your role. Give me a suggestion for a main course for today's lunch.
With this suggestion, I also want a table with two columns where each row in the table contains an ingredient from the main course.
The first column in the table is the name of the ingredient.
The second column of the table is the number of grams of that ingredient needed for one person.
Do not give the recipe for preparing the main course.
"""
chat_completion(prompt)

Main Course Suggestion: Quinoa and Black Bean Salad

| Ingredient | Quantity (grams) |
|------------|-----------------|
| Quinoa     | 185             |
| Black Beans| 130             |
| Corn       | 80              |
| Red Bell Pepper | 75        |
| Avocado    | 150             |
| Lime       | 30              |
| Olive Oil  | 15              |
| Cilantro   | 10              |
| Salt       | 5               |
| Pepper     | 2               |


## Thinking Step by Step

In [9]:
# GPT-4不擅长计算，它将数字视为token，没有数学逻辑，当然可以采用计算器插件的形式来弥补，我们先来看一下直接作为Prompt会是什么效果？
prompt = "How much is 369 * 1235?"
chat_completion(prompt)

455415


答案显然是不正确的，正确答案是455715.

In [12]:
# "Let's think step by step"技术被称为zero-shot-CoT，下面来看看使用这种技术的效果
prompt = "How much is 369 * 1235? Let's think step by step."
chat_completion(prompt)


First, multiply 369 by 5. That gives us 1845.

Next, multiply 369 by 30 (which is the same as multiplying by 3 and then adding a zero at the end). That gives us 11070.

Then, multiply 369 by 2000 (which is the same as multiplying by 2 and then adding three zeros at the end). That gives us 738000.

Finally, add all these results together: 1845 + 11070 + 738000 = 754915.

So, 369 * 1235 = 754915.


尽管答案是错误的，但是确实是将任务进行了分解，只是过程中部分代码错误了

## Few Shot Learning

In [13]:
# 本例中，我们要求LLM将特定的单词转换为表情符号，使用少样本学习，就变的比较容易了
prompt = """
I go home --> 😊 go 🏠
my dog is sad --> my 🐶 is 😞
I run fast --> 😊 run ⚡
I love my wife --> 😊 ❤️ my wife
the girl plays with the ball --> the 👧 🎮 with the 🏀
The boy writes a letter to a girl -->
"""
chat_completion(prompt)


The 👦 ✍️ a 💌 to a 👧
